# Introducing the Dataset

In [1]:
# Some initialization magic
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import collections

# Let's load up the data
import json

with open("scrapyproject/results.json") as fin:
    tables = [json.loads(line) for line in fin]

In [2]:
# Here's what it looks like. 
# Each line corresponds to a table scraped from the Central Electoral Committee website.
# Tables are dictionaries.
sorted(tables[0].keys())

['column_headers',
 'data',
 'data_type',
 'md5',
 'region',
 'region_ik',
 'region_ik_long',
 'row_headers',
 'timestamp',
 'url']

In [3]:
#
# There are four kinds of data.
#     results_tik: Results for a territory IK (electoral committee)
#     results_uik: Results for an individual polling station
#     turnout_tik: Turnout for a territory IK
#     turnout_uik: Turnout for an individual polling station
#
import collections
collections.Counter([t['data_type'] for t in tables])

Counter({'results_tik': 87,
         'results_uik': 2762,
         'turnout_tik': 86,
         'turnout_uik': 2753})

In [4]:
# region contains the name of the region.
a_table = [t for t in tables if t["data_type"] == "results_tik"][0]
a_table["region"]

'Республика Адыгея (Адыгея)'

In [5]:
# region_ik contains the number of the regional election committee (окружная избирательная коммиссия).
# TODO: why is this empty?
a_table["region_ik"]

''

In [6]:
# region_ik_long is the full name of the local election committee
a_table["region_ik_long"]

'Республика Адыгея (Адыгея)'

In [7]:
# url corresponds to the URL from which the data was scraped from
a_table["url"]

'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1000022&tvd=100100031793851&vrn=100100031793505&region=0&global=true&sub_region=0&prver=0&pronetvd=null&vibid=100100031793851&type=227'

In [8]:
# the timestamp shows the date and time that the data was scraped
a_table["timestamp"]

'2018-03-11T10:21:07.509412+00:00'

In [9]:
# md5 corresponds to the MD5 hash of the HTML that was fetched from the URL at the time of scraping
import requests
import hashlib
r = requests.get(a_table["url"])
assert hashlib.md5(r.content).hexdigest() == a_table["md5"], "MD5 mismatch, perhaps data is out of date?"

In [10]:
# Each table, regardless of data_type, contains data in rows and columns.
# The label for each row and column is contained in "row_headers" and "column_headers", respectively.
# For turnout tables, the data looks like this:
# The row headers in this case are the names of the local electoral committees (участковая избирательная коммиссия).
# The column headers are the times at which turnout was reported.
turnout_table = [t for t in tables if t["data_type"] == "turnout_tik"][0]
print(turnout_table["row_headers"])
print(turnout_table["column_headers"])

['ВСЕГО, в том числе', 'Адыгейская', 'Гиагинская', 'Кошехабльская', 'Красногвардейская', 'Майкопская районная', 'Майкопская городская', 'Тахтамукайская', 'Теучежская', 'Шовгеновская']
['10:00', '12:00', '15:00', '18:00']


In [11]:
# For result tables, the data looks like this:
results_table = [t for t in tables if t["data_type"] == "results_uik"][0]
print(results_table["row_headers"][:5])  # snipped for brevity
print(results_table["column_headers"][:5]) # snipped for brevity

['Число избирателей, включенных в список избирателей', 'Число избирательных бюллетеней, полученных участковой избирательной комиссией', 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно', 'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования', 'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования']
['Сумма', 'УИК №186', 'УИК №187', 'УИК №188', 'УИК №189']


In [12]:
# Rows correspond to a certain measurement, the columns correspond to the local electoral committees.
# The list of candidates is the same across the entire country.
print(results_table["row_headers"][18:24])  # snipped for brevity

['Жириновский Владимир Вольфович', 'Зюганов Геннадий Андреевич', 'Миронов Сергей Михайлович', 'Прохоров Михаил Дмитриевич', 'Путин Владимир Владимирович']


In [13]:
#
# The *_uik tables contain the same row headers as their regular counterparts.
# They just show finer-grained information.
#
uik_table = [t for t in tables if t["data_type"] == "results_uik"][0]
assert uik_table["row_headers"] == results_table["row_headers"]
uik_table["column_headers"][:5]  # snipped for brevity

['Сумма', 'УИК №186', 'УИК №187', 'УИК №188', 'УИК №189']

In [14]:
# They also include the territory electoral committe (территориальная избирательная комиссия)
# TODO: this also looks broken
uik_table["territory_ik"]

''

In [15]:
# The data itself can be obtained by indexing into the data matrix.
results_table["data"][0][0]

50984.0

# Checking the Dataset

In [16]:
# Make sure we have complete data for each
counter = collections.Counter(table["data_type"] for table in tables)
counter

Counter({'results_tik': 87,
         'results_uik': 2762,
         'turnout_tik': 86,
         'turnout_uik': 2753})

In [17]:
# TODO

# Summary

In [18]:
# The number of OIK (region electoral committees), TIK (territory electoral committees),
# and UIK (spot electoral committees) per region
import csv
import sys

writer = csv.writer(sys.stdout, delimiter="|")
writer.writerow(["region", "oik", "tik", "uik"])

results = [t for t in tables if t["data_type"] == "results_tik"]
results_uik = [t for t in tables if t["data_type"] == "results_uik"]
oik_counter = collections.Counter(t["region"] for t in results)

for region, num_oik in sorted(oik_counter.items()):
    region_tables = [t for t in results if t["region"] == region]
    num_tik = sum([(len(t["column_headers"]) - 1) for t in region_tables])
    
    region_tables_uik = [t for t in results_uik if t["region"] == region]
    num_uik = sum([(len(t["column_headers"]) - 1) for t in region_tables_uik])
    
    writer.writerow([region, num_oik, num_tik, num_uik])

region|oik|tik|uik
Алтайский край|1|75|1862
Амурская область|1|29|783
Архангельская область|1|31|984
Астраханская область|1|16|586
Белгородская область|1|22|1250
Брянская область|1|34|1124
Владимирская область|1|23|940
Волгоградская область|1|46|1653
Вологодская область|1|28|1040
Воронежская область|1|39|1680
Город Байконур (Республика Казахстан)|1|0|0
Город Москва|1|125|3386
Город Санкт-Петербург|1|30|1937
Еврейская автономная область|1|6|144
Забайкальский край|1|38|955
Ивановская область|1|30|717
Иркутская область|1|45|1925
Кабардино-Балкарская Республика|1|13|356
Калининградская область|1|24|0
Калужская область|1|28|723
Камчатский край|1|14|323
Карачаево-Черкесская Республика|1|12|246
Кемеровская область|1|47|1718
Кировская область|1|48|1211
Костромская область|1|30|623
Краснодарский край|1|59|2713
Красноярский край|1|70|2174
Курганская область|1|27|1175
Курская область|1|35|1153
Ленинградская область|1|18|989
Липецкая область|1|23|875
Магаданская область|1|10|103
Московская область